In [1]:
import fastf1
import pandas as pd
import os
import fastf1.events as f1events

In [2]:
#create folder
cache_dir = os.path.expanduser("~/f1_cache")  
os.makedirs(cache_dir, exist_ok=True)
#enable cache
fastf1.Cache.enable_cache(cache_dir)

In [ ]:
fastf1.Cache.enable_cache(cache_dir)

#set year range
YEARS = range(2020, 2026)  

all_laps = []

for year in YEARS:
    schedule = fastf1.get_event_schedule(year, include_testing=False)
    non_sprint_events = schedule[schedule['EventFormat'] != 'sprint']

    for i, event in non_sprint_events.iterrows():
        round_num = event['RoundNumber']
        event_name = event['EventName']

        try:
            session = fastf1.get_session(year, round_num, 'R') #races only
            print(f"Loading {year} Round {round_num} - {event_name}")
            session.load()  
            laps = session.laps.copy()
            laps['RaceName'] = event_name
            laps['Year'] = year
            laps['PostRegulation'] = int(year >= 2022)

            #merge weather data
            weather = session.weather_data.copy()
            laps['LapStartTimeSec'] = laps['LapStartTime'].dt.total_seconds()
            weather['TimeSec'] = weather['Time'].dt.total_seconds()

            laps = pd.merge_asof(
                laps.sort_values('LapStartTimeSec'),
                weather[['TimeSec', 'AirTemp', 'TrackTemp', 'WindSpeed', 'Humidity']],
                left_on='LapStartTimeSec',
                right_on='TimeSec',
                direction='nearest'
            )

            all_laps.append(laps)
        except Exception as e:
            print(f"Skipped {event_name} ({year}) due to error: {e}")

#combine all datasets to one & export
if all_laps:
    final_df = pd.concat(all_laps, ignore_index=True)
    final_df.to_csv('laps_2020_2025.csv', index=False)